# Enet using deconvo genes from Bayes Prism (mirroring feature engineering as closely as possible to original paper)

In [ ]:
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split

In [64]:
# import data
ann_tcga = pd.read_csv('../data/toga.breast.brca.status.txt', sep='\t', index_col=0)
tcga_brca = pd.read_csv('../data/tcga.brca.rnaseq.unstranded.fpkm.counts.matrix.txt', sep='\t', index_col=0)
deconvo = pd.read_csv('../data/Deconvo2.csv',  index_col=0)
hrd_scores = pd.read_excel('../data/tcga.hrdscore.xlsx')

In [65]:
# ann_tcga[ann_tcga['event.PALB2'].ne('0')]
# ann_tcga[ann_tcga['event.RAD51C'].ne('0')]
ann_tcga = ann_tcga[~ann_tcga['event.RAD51C'].ne('0')]
ann_tcga = ann_tcga[~ann_tcga['event.PALB2'].ne('0')]
ann_tcga = ann_tcga[ann_tcga['event.BRCA1'] != '1']

In [66]:
ann_tcga.index = ann_tcga.index.str.replace('.', '-', regex=False)
tcga_brca = tcga_brca.set_index('Case ID')
print(f"annotated shape{ann_tcga.shape}")
print(f"rna-seq shape{tcga_brca.shape}")

annotated shape(962, 33)
rna-seq shape(1231, 60662)


In [ ]:
common_indices = ann_tcga.index.intersection(tcga_brca.index)

# Filter DataFrames to keep only rows with common indices
ann_tcga = ann_tcga.loc[common_indices]
tcga_brca = tcga_brca.loc[common_indices]
print(f"rna-seq shape{tcga_brca.shape}")

rna-seq shape(1070, 60662)


In [33]:
tcga_brca = tcga_brca[tcga_brca['Sample Type'] == 'Primary Tumor']

In [71]:
deconvo.index = deconvo.index.map(lambda x: x[:12])

In [ ]:
common_indices1 = deconvo.index.intersection(tcga_brca.index)
tcga_brca.loc[common_indices1]

,Sample ID,Sample Type,ENSG00000000003.15|TSPAN6|protein_coding,ENSG00000000005.6|TNMD|protein_coding,ENSG00000000419.13|DPM1|protein_coding,ENSG00000000457.14|SCYL3|protein_coding,ENSG00000000460.17|C1orf112|protein_coding,ENSG00000000938.13|FGR|protein_coding,ENSG00000000971.16|CFH|protein_coding,ENSG00000001036.14|FUCA2|protein_coding,...,ENSG00000288661.1|AL451106.1|protein_coding,ENSG00000288662.1|AC092910.4|lncRNA,ENSG00000288663.1|AC073611.1|lncRNA,ENSG00000288665.1|AC136977.1|lncRNA,ENSG00000288667.1|AC078856.1|lncRNA,ENSG00000288669.1|AC008763.4|protein_coding,ENSG00000288670.1|AL592295.6|lncRNA,ENSG00000288671.1|AC006486.3|protein_coding,ENSG00000288674.1|AL391628.1|protein_coding,ENSG00000288675.1|AP006621.6|protein_coding
TCGA-A1-A0SB,TCGA-A1-A0SB-01A,Primary Tumor,19.6132,8.7767,21.2168,3.1036,1.2107,0.7039,24.3075,5.9890,...,0.0,0.0000,0.0903,0.0,0.0000,0.0,2.5875,0.0,0.0245,0.3319
TCGA-A1-A0SD,TCGA-A1-A0SD-01A,Primary Tumor,9.2472,0.5736,38.9326,3.5182,1.0637,2.0629,10.4956,20.2043,...,0.0,0.0000,0.0743,0.0,0.0000,0.0,3.2987,0.0,0.0035,0.1915
TCGA-A1-A0SE,TCGA-A1-A0SE-01A,Primary Tumor,18.7699,0.2720,21.6648,4.2998,3.6655,1.0240,5.9898,7.6596,...,0.0,0.0000,0.0470,0.0,0.0000,0.0,5.7125,0.0,0.0052,0.2988
TCGA-A1-A0SF,TCGA-A1-A0SF-01A,Primary Tumor,42.0744,0.1293,28.5348,3.8915,1.6092,3.6183,7.2606,10.6490,...,0.0,0.0000,0.0707,0.0,0.0000,0.0,5.0936,0.0,0.0088,0.4040
TCGA-A1-A0SG,TCGA-A1-A0SG-01A,Primary Tumor,11.9388,0.0492,26.2437,2.6332,1.2095,1.9633,6.9818,13.6613,...,0.0,0.0000,0.0541,0.0,0.0000,0.0,4.7087,0.0,0.0075,0.5075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-GM-A3NY,TCGA-GM-A3NY-01A,Primary Tumor,10.6640,1.3452,20.1907,4.1455,1.1694,2.2867,3.1783,11.3763,...,0.0,0.0000,0.2248,0.0,0.0000,0.0,3.4381,0.0,0.0399,0.5814
TCGA-B6-A3ZX,TCGA-B6-A3ZX-01A,Primary Tumor,10.6974,0.0477,31.7794,3.8813,2.2124,6.4795,5.8249,9.4418,...,0.0,0.0000,0.1358,0.0,0.0000,0.0,2.6571,0.0,0.0238,0.3141
TCGA-EW-A3E8,TCGA-EW-A3E8-01B,Primary Tumor,8.5772,3.5775,29.1650,3.4800,1.6536,4.3762,5.5199,11.0641,...,0.0,0.0000,0.1719,0.0,0.0000,0.0,2.3208,0.0,0.0143,0.4828
TCGA-D8-A3Z5,TCGA-D8-A3Z5-01A,Primary Tumor,32.0458,1.9986,20.0412,5.7002,1.5287,2.3551,4.5446,7.9185,...,0.0,0.0000,0.2426,0.0,0.0000,0.0,4.9486,0.0,0.0460,0.3402


In [ ]:

# tcga_brca.loc[tcga_brca.index.difference(deconvo.index)]
# deconvo.loc[deconvo.index.difference(tcga_brca.index)]

,Sample ID,Sample Type,ENSG00000000003.15|TSPAN6|protein_coding,ENSG00000000005.6|TNMD|protein_coding,ENSG00000000419.13|DPM1|protein_coding,ENSG00000000457.14|SCYL3|protein_coding,ENSG00000000460.17|C1orf112|protein_coding,ENSG00000000938.13|FGR|protein_coding,ENSG00000000971.16|CFH|protein_coding,ENSG00000001036.14|FUCA2|protein_coding,...,ENSG00000288661.1|AL451106.1|protein_coding,ENSG00000288662.1|AC092910.4|lncRNA,ENSG00000288663.1|AC073611.1|lncRNA,ENSG00000288665.1|AC136977.1|lncRNA,ENSG00000288667.1|AC078856.1|lncRNA,ENSG00000288669.1|AC008763.4|protein_coding,ENSG00000288670.1|AL592295.6|lncRNA,ENSG00000288671.1|AC006486.3|protein_coding,ENSG00000288674.1|AL391628.1|protein_coding,ENSG00000288675.1|AP006621.6|protein_coding
TCGA-A1-A0SB,TCGA-A1-A0SB-01A,Primary Tumor,19.6132,8.7767,21.2168,3.1036,1.2107,0.7039,24.3075,5.9890,...,0.0,0.0000,0.0903,0.0,0.0,0.0,2.5875,0.0,0.0245,0.3319
TCGA-A1-A0SD,TCGA-A1-A0SD-01A,Primary Tumor,9.2472,0.5736,38.9326,3.5182,1.0637,2.0629,10.4956,20.2043,...,0.0,0.0000,0.0743,0.0,0.0,0.0,3.2987,0.0,0.0035,0.1915
TCGA-A1-A0SE,TCGA-A1-A0SE-01A,Primary Tumor,18.7699,0.2720,21.6648,4.2998,3.6655,1.0240,5.9898,7.6596,...,0.0,0.0000,0.0470,0.0,0.0,0.0,5.7125,0.0,0.0052,0.2988
TCGA-A1-A0SF,TCGA-A1-A0SF-01A,Primary Tumor,42.0744,0.1293,28.5348,3.8915,1.6092,3.6183,7.2606,10.6490,...,0.0,0.0000,0.0707,0.0,0.0,0.0,5.0936,0.0,0.0088,0.4040
TCGA-A1-A0SG,TCGA-A1-A0SG-01A,Primary Tumor,11.9388,0.0492,26.2437,2.6332,1.2095,1.9633,6.9818,13.6613,...,0.0,0.0000,0.0541,0.0,0.0,0.0,4.7087,0.0,0.0075,0.5075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-OL-A6VR,TCGA-OL-A6VR-01A,Primary Tumor,14.1613,0.0996,28.4775,4.2414,1.1767,1.6086,3.4413,11.2886,...,0.0,0.2723,0.1032,0.0,0.0,0.0,2.9473,0.0,0.0136,0.2917
TCGA-PE-A5DC,TCGA-PE-A5DC-01A,Primary Tumor,10.8397,0.0711,26.2361,2.6312,1.8876,1.9111,3.6606,15.6981,...,0.0,0.0000,0.2504,0.0,0.0,0.0,5.0286,0.0,0.0087,0.3248
TCGA-PE-A5DD,TCGA-PE-A5DD-01A,Primary Tumor,7.4238,0.4234,24.4818,4.9614,2.5460,3.5407,8.5619,13.1997,...,0.0,0.0000,0.3158,0.0,0.0,0.0,1.3558,0.0,0.0468,0.3423
TCGA-PE-A5DE,TCGA-PE-A5DE-01A,Primary Tumor,2.3539,0.1792,39.0309,6.2156,2.2679,7.8430,8.5525,7.3712,...,0.0,0.0000,0.1182,0.0,0.0,0.0,2.6750,0.0,0.0250,0.3149


In [ ]:
hrd_scores = hrd_scores.set_index('sample')

hrd_scores = hrd_scores.loc[hrd_scores.index.intersection(deconvo.index)]
labels_df = hrd_scores['HRD-sum'].apply(lambda x: 'HRD' if x >= 79 else 'HR-Proficient')
labels_df

TCGA-3C-AAAU    HR-Proficient
TCGA-3C-AALI    HR-Proficient
TCGA-3C-AALJ    HR-Proficient
TCGA-3C-AALK    HR-Proficient
TCGA-4H-AAAK    HR-Proficient
                    ...      
TCGA-W8-A86G    HR-Proficient
TCGA-WT-AB41    HR-Proficient
TCGA-XX-A89A    HR-Proficient
TCGA-Z7-A8R5    HR-Proficient
TCGA-Z7-A8R6    HR-Proficient
Name: HRD-sum, Length: 933, dtype: object

In [ ]:
# Ensure your labels are in the correct format
# If labels_df is a DataFrame, convert it to a 1D array
labels = labels_df.squeeze()
features_df = deconvo

# Initialize a list to store coefficients from each iteration
coefficients_list = []

# Split the data into training and test sets (1/3 test split)
X_train, X_test, y_train, y_test = train_test_split(
    features_df, labels, test_size=1/3, stratify=labels, random_state=123
)

# Run 1000 iterations
for iteration in range(1000):
    print(f"Iteration {iteration + 1} of 1000")
    
    
    # Define the LogisticRegressionCV model
    logreg_cv = LogisticRegressionCV(
        Cs=10,  # Number of Cs to try; you can specify an array of Cs if desired
        cv=10,  # Tenfold cross-validation
        penalty='elasticnet',
        solver='saga',  # Solver that supports elastic net penalty
        # multi_class='multinomial',
        multi_class='auto',
        l1_ratios=[0.25],  # Alpha = 0.25
        max_iter=1000,
        n_jobs=-1,  # Utilize all processors
        random_state=123*iteration
    )
    
    # Fit the model
    logreg_cv.fit(X_train, y_train)
    
    # Get coefficients at the best C (lambda.min)
    coef = logreg_cv.coef_  # Shape: (n_classes, n_features)
    
    # Append the coefficients to the list
    coefficients_list.append(coef)

# Convert the list of coefficients to a NumPy array
coefficients_array = np.array(coefficients_list)  # Shape: (1000, n_classes, n_features)

# If you want to average the coefficients over all iterations:
average_coefficients = np.mean(coefficients_array, axis=0)

# Optionally, create a DataFrame for better visualization
feature_names = features_df.columns
class_labels = logreg_cv.classes_
coefficients_df = pd.DataFrame(
    average_coefficients,
    index=class_labels,
    columns=feature_names
)


Iteration 1 of 1000


/Users/danirahman/miniforge3/envs/hrdclass/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1895: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, binary problems will be fit as proper binary  logistic regression models (as if multi_class='ovr' were set). Leave it to its default value to avoid this warning.
  warnings.warn(


In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import StratifiedKFold

# Convert your pandas DataFrames to NumPy arrays
X = features_df.values
y = labels_df.values.ravel()  # Ensure y is a 1D array

# Set up the cross-validation strategy
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Initialize the LogisticRegressionCV model
model = LogisticRegressionCV(
    Cs=10,                      # Number of regularization strengths
    cv=cv,                      # Cross-validation strategy
    penalty='elasticnet',       # Elastic net penalty
    solver='saga',              # Solver that supports elastic net penalty
    l1_ratios=[0.25],           # Equivalent to alpha=0.25 in R
    multi_class='multinomial',  # Multinomial logistic regression
    max_iter=1000,              # Maximum number of iterations
    n_jobs=-1,                  # Use all available CPU cores
    scoring='accuracy',         # Evaluation metric
    refit=True,                 # Refit the model on the entire dataset
    random_state=42             # Ensure reproducibility
)

# Fit the model
model.fit(X, y)
